Trong hướng dẫn này, bạn sẽ tìm hiểu **data leakage** (*rò rỉ dữ liệu*) là gì và cách ngăn chặn nó.  

Nếu bạn không biết cách ngăn chặn *data leakage*, vấn đề này sẽ xuất hiện thường xuyên và làm hỏng mô hình của bạn theo những cách tinh vi và nguy hiểm.  

Vì vậy, đây là một trong những khái niệm quan trọng nhất đối với các *data scientist* thực hành.

# Giới thiệu (# Introduction #)

**Data leakage** (hay còn gọi là **leakage**) xảy ra khi dữ liệu huấn luyện chứa thông tin về biến mục tiêu (*target*), nhưng thông tin tương tự sẽ không có sẵn khi mô hình được sử dụng để dự đoán.  

Điều này dẫn đến mô hình có hiệu suất cao trên tập huấn luyện (và có thể cả tập kiểm định), nhưng lại hoạt động kém trong môi trường thực tế.

Nói cách khác, *leakage* khiến mô hình có vẻ chính xác cho đến khi bạn bắt đầu sử dụng nó để đưa ra quyết định — và khi đó, mô hình trở nên rất không chính xác.

Có hai loại *leakage* chính: **target leakage** và **train-test contamination**.

### Target leakage

**Target leakage** xảy ra khi tập đặc trưng (*predictors*) bao gồm dữ liệu mà tại thời điểm thực hiện dự đoán, dữ liệu đó chưa có sẵn.  

Điều quan trọng là phải xem xét *target leakage* theo _trình tự thời gian hoặc thứ tự mà dữ liệu xuất hiện_, chứ không chỉ đơn thuần là một biến giúp dự đoán tốt.

Một ví dụ sẽ giúp bạn hiểu rõ hơn.  

Giả sử bạn muốn dự đoán ai sẽ bị viêm phổi (*pneumonia*). Một số hàng đầu tiên của tập dữ liệu thô trông như sau:

| got_pneumonia | age | weight |  male | took_antibiotic_medicine | ... |
|:-------------:|:---:|:------:|:-----:|:------------------------:|-----|
|     False     |  65 |   100  | False |           False          | ... |
|     False     |  72 |   130  |  True |           False          | ... |
|      True     |  58 |   100  | False |           True           | ... |

Người bệnh uống kháng sinh *sau khi* bị viêm phổi để hồi phục. Dữ liệu thô thể hiện một mối quan hệ mạnh giữa các cột, nhưng biến `took_antibiotic_medicine` thường thay đổi *sau* khi giá trị `got_pneumonia` được xác định.  

Đây chính là *target leakage*.

Mô hình sẽ nhận thấy rằng bất kỳ ai có giá trị `False` trong cột `took_antibiotic_medicine` đều không bị viêm phổi. Vì dữ liệu kiểm định có nguồn gốc tương tự như dữ liệu huấn luyện, mô hình sẽ lặp lại mô hình này và có điểm số kiểm định (hoặc *cross-validation*) rất cao.

Tuy nhiên, khi triển khai thực tế, mô hình sẽ rất không chính xác vì ngay cả những bệnh nhân sẽ mắc viêm phổi cũng chưa uống kháng sinh tại thời điểm chúng ta cần dự đoán về sức khỏe của họ.

Để ngăn chặn loại *data leakage* này, bất kỳ biến nào được cập nhật (hoặc tạo) sau khi giá trị *target* được xác định nên được loại trừ.

![tut7_leakydata](https://storage.googleapis.com/kaggle-media/learn/images/y7hfTYe.png)

### Train-Test Contamination

Một loại *leakage* khác xảy ra khi bạn không cẩn thận trong việc tách biệt dữ liệu huấn luyện và dữ liệu kiểm định.  

Nhớ rằng mục tiêu của tập kiểm định là đo lường cách mô hình hoạt động trên dữ liệu mà nó chưa từng thấy trước đó.  

Bạn có thể vô tình làm sai lệch quy trình này nếu dữ liệu kiểm định ảnh hưởng đến quá trình tiền xử lý. Điều này đôi khi được gọi là **train-test contamination** (*ô nhiễm train-test*).

Ví dụ, hãy tưởng tượng bạn thực hiện tiền xử lý (như huấn luyện một *imputer* để điền giá trị bị thiếu) trước khi gọi hàm [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).  

Kết quả?  

Mô hình của bạn có thể đạt điểm kiểm định cao, làm bạn tin tưởng vào nó, nhưng khi triển khai thực tế lại hoạt động kém.

Vấn đề là bạn đã vô tình sử dụng thông tin từ tập kiểm định hoặc tập kiểm tra (*test set*) trong quá trình huấn luyện, nên mô hình có thể hoạt động tốt trên tập dữ liệu đó nhưng lại không tổng quát hóa được trên dữ liệu mới.  

Vấn đề này càng trở nên tinh vi (*subtle*) (và nguy hiểm hơn) khi bạn thực hiện các kỹ thuật trích xuất đặc trưng (*feature engineering*) phức tạp hơn.

Nếu bạn chia dữ liệu thành tập huấn luyện và kiểm định theo cách đơn giản (*train-test split*), hãy loại bỏ hoàn toàn tập kiểm định khỏi mọi bước *fitting*, bao gồm cả các bước *fitting* trong quá trình tiền xử lý.  

Điều này sẽ dễ dàng hơn nếu bạn sử dụng *pipelines* trong *scikit-learn*.  

Khi sử dụng *cross-validation*, việc đảm bảo tất cả các bước tiền xử lý được thực hiện bên trong *pipeline* lại càng quan trọng hơn!

# Ví dụ (# Example #)

Trong ví dụ này, bạn sẽ học cách phát hiện và loại bỏ *target leakage*.

Chúng ta sẽ sử dụng một tập dữ liệu về đơn xin cấp thẻ tín dụng (*credit card applications*) và bỏ qua phần thiết lập dữ liệu cơ bản.  

Kết quả cuối cùng là thông tin về từng đơn xin cấp thẻ được lưu trong một *DataFrame* `X`.  

Chúng ta sẽ sử dụng nó để dự đoán xem đơn xin cấp thẻ có được chấp nhận hay không trong một *Series* `y`.

In [3]:

import pandas as pd

# Read the data
data = pd.read_csv('AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

# Select target
y = data.card

# Select predictors
X = data.drop(['card'], axis=1)

print("Number of rows in the dataset:", X.shape[0])
X.head()

Number of rows in the dataset: 1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


Vì đây là một tập dữ liệu nhỏ, chúng ta sẽ sử dụng *cross-validation* để đảm bảo đo lường chính xác chất lượng mô hình.

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there is no preprocessing, we don't need a pipeline (used anyway as best practice!)
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

Cross-validation accuracy: 0.981810


Với kinh nghiệm, bạn sẽ nhận ra rằng rất hiếm khi tìm thấy các mô hình có độ chính xác lên đến 98%.  

Điều này có thể xảy ra, nhưng nó đủ hiếm để chúng ta cần kiểm tra dữ liệu kỹ lưỡng hơn để phát hiện *target leakage*.

Dưới đây là bản tóm tắt về tập dữ liệu, bạn cũng có thể tìm thấy thông tin này trong tab dữ liệu:

- **`card`**: 1 nếu đơn xin cấp thẻ tín dụng được chấp nhận, 0 nếu bị từ chối  
- **`reports`**: Số lượng báo cáo tiêu cực nghiêm trọng (*major derogatory reports*)  
- **`age`**: Tuổi tính theo năm cộng với phần lẻ của năm  
- **`income`**: Thu nhập hàng năm (chia cho 10,000)  
- **`share`**: Tỷ lệ chi tiêu hàng tháng trên thẻ tín dụng so với thu nhập hàng năm  
- **`expenditure`**: Chi tiêu trung bình hàng tháng trên thẻ tín dụng  
- **`owner`**: 1 nếu sở hữu nhà, 0 nếu thuê nhà  
- **`selfempl`**: 1 nếu tự kinh doanh (*self-employed*), 0 nếu không  
- **`dependents`**: 1 + số lượng người phụ thuộc  
- **`months`**: Số tháng sinh sống tại địa chỉ hiện tại  
- **`majorcards`**: Số lượng thẻ tín dụng lớn đang sở hữu  
- **`active`**: Số lượng tài khoản tín dụng đang hoạt động  

Một số biến có vẻ đáng ngờ.  

Ví dụ, biến **`expenditure`** có ý nghĩa là chi tiêu trên thẻ tín dụng này hay trên các thẻ đã sử dụng trước khi nộp đơn?

Tại thời điểm này, việc so sánh dữ liệu cơ bản có thể rất hữu ích:

In [5]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


Như đã thấy ở trên, tất cả những người không được cấp thẻ đều không có khoản chi tiêu nào, trong khi chỉ có 2% số người được cấp thẻ không có khoản chi tiêu.  

Do đó, không có gì ngạc nhiên khi mô hình của chúng ta đạt độ chính xác cao.  

Tuy nhiên, đây dường như là một trường hợp *target leakage*, vì biến **`expenditure`** có thể đại diện cho *chi tiêu trên thẻ mà họ vừa đăng ký*.

Vì biến **`share`** được tính toán một phần dựa trên **`expenditure`**, nên nó cũng cần bị loại bỏ.  

Các biến **`active`** và **`majorcards`** ít rõ ràng hơn, nhưng theo mô tả, chúng có vẻ đáng lo ngại.  

Trong hầu hết các tình huống, nếu bạn không thể truy vấn trực tiếp những người tạo ra dữ liệu để xác minh ý nghĩa chính xác, thì nên loại bỏ các biến này để đảm bảo an toàn.

Chúng ta có thể huấn luyện mô hình mà không có *target leakage* như sau:

In [6]:
# Drop leaky predictors from dataset
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Evaluate the model with leaky predictors removed
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-val accuracy: %f" % cv_scores.mean())

Cross-val accuracy: 0.830162


Độ chính xác của mô hình hiện tại thấp hơn đáng kể, điều này có thể gây thất vọng.  

Tuy nhiên, chúng ta có thể kỳ vọng mô hình sẽ dự đoán đúng khoảng 80% khi áp dụng cho các đơn xin cấp thẻ mới.  

Trong khi đó, mô hình có *leakage* có thể hoạt động kém hơn đáng kể khi áp dụng vào thực tế, mặc dù đạt điểm số cao trong *cross-validation*.

# Kết luận (# Conclusion #)

*Data leakage* có thể dẫn đến những sai lầm trị giá hàng triệu đô la trong nhiều ứng dụng *data science*.  

Việc tách biệt cẩn thận tập huấn luyện và tập kiểm định có thể ngăn chặn *train-test contamination*, và việc sử dụng *pipelines* có thể giúp triển khai quy trình này một cách hệ thống.  

Tương tự, sự kết hợp giữa thận trọng, tư duy logic và khám phá dữ liệu có thể giúp phát hiện *target leakage*.

# Bước tiếp theo? (# What's next? #)

Khái niệm này có thể vẫn còn trừu tượng.  

Hãy thử suy nghĩ về các ví dụ trong **[bài tập này](https://www.kaggle.com/kernels/fork/3370270)** để rèn luyện kỹ năng phát hiện *target leakage* và *train-test contamination*!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intermediate-machine-learning/discussion) to chat with other learners.*